# Optimizing performance of Julia code
Przemysław Szufel

<a class="anchor" id="toc"></a>
## Table of content

1. [Avoid global variables](#globals)
2. [Avoid abstract lists and structs](#lists)
3. [Do not change the type of a variable within a function](#fvariable)
4. [Mitigate type uncertainty with barrier functions](#barrier)
5. [Remember about column-major layout of matrices](#column)
6. [Prealocate, use views, vectorize](#views)
7. [Check type stability with the `@code_warntype` macro](#warntype)
    


<p style="font-size:8pt">
    Reference: several examples below are inspired by
https://docs.julialang.org/en/v1/manual/performance-tips/
 </p>

<a class="anchor" id="globals"></a>
### Avoid global variables
---- [Return to table of contents](#toc) ---

In [2]:
x = rand(10_000)  #global

function loop_over_global()
    s = 0.0
    for i in x
        s += i
    end
    return s
end

function loop_over_x(x)
    s = 0.0
    for i in x
        s += i
    end
    return s
end

function loop_over_global_type_assert()
    s = 0.0
    for i in x::Vector{Float64}
        s += i
    end
    return s
end

@time loop_over_global()
@time loop_over_global()
@time loop_over_x(x)
@time loop_over_x(x)
@time loop_over_global_type_assert()
@time loop_over_global_type_assert()

  0.004230 seconds (39.49 k allocations: 773.281 KiB)
  0.003697 seconds (39.49 k allocations: 773.281 KiB)
  0.024790 seconds (2.10 k allocations: 136.422 KiB, 99.76% compilation time)
  0.000023 seconds (1 allocation: 16 bytes)
  0.000021 seconds
  0.000026 seconds


5028.336015998785

In [2]:
@time loop_over_global_type_assert()
@time loop_over_global_type_assert()

  0.000028 seconds
  0.000017 seconds


4998.554629564902

In [4]:
using BenchmarkTools
BenchmarkTools.@btime  loop_over_global()
@btime loop_over_x($x)

  748.500 μs (39490 allocations: 773.28 KiB)
  6.220 μs (0 allocations: 0 bytes)


5028.336015998785

In [5]:
BenchmarkTools.@benchmark loop_over_global()

BenchmarkTools.Trial: 2879 samples with 1 evaluation.
 Range (min … max):  739.500 μs … 28.563 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.593 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.719 ms ±  1.066 ms  ┊ GC (mean ± σ):  1.90% ± 6.86%

    ▂▂▄▅▆▇▆▆▆▆▆▆▇█▇▄▄▂▂▁ ▁   ▁                                 ▂
  █████████████████████████████████▇███▇██▇▇█▇▇▆▇▇▆▇▆▇▅▅▄▆▄▄▁▅ █
  740 μs        Histogram: log(frequency) by time      4.44 ms <

 Memory estimate: 773.28 KiB, allocs estimate: 39490.

In [6]:
BenchmarkTools.@benchmark loop_over_x($x)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):   3.975 μs … 46.600 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.537 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):    9.509 μs ±  3.873 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▂  ▇▄▂ ▂▁    ▁▁▃▃▁ ▃ ▁       █ ▃ ▂ ▂ ▅  ▃  ▃  ▃            ▂
  ██▇███████▆██▇█████▆█▇█▇▅▅▇▆▅▅█████▇█▇█▇▇█▆██▄▆█▃▅▆▃▁▄▅▁▄▃▆ █
  3.98 μs      Histogram: log(frequency) by time      18.8 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [6]:
const zzz = x
function loop_over_global_zz()
    s = 0.0
    for i in zzz  
        s += i
    end
    return s
end

@btime loop_over_global_zz()

  6.220 μs (0 allocations: 0 bytes)


5028.336015998785

<a class="anchor" id="lists"></a>
### Avoid abstract lists and  structs
---- [Return to table of contents](#toc) ---

In [8]:
# do not use abstract lists
function addelems_and_sum(arr::AbstractVector)
    for i in 1.0:100.0
        push!(arr,i)
    end
    sum(arr)
end
myarr = [] # DO NOT DO IT!  Any[]
myarr1 = Real[]  # DO NOT DO IT !
myarr2 = Float64[] # Correct
myarr3 = Union{Float64, ComplexF32}[]# If you really need this type flexibility use small unions
myarr4 = Union{Float64, Missing}[]




Union{Missing, Float64}[]

In [9]:
@btime addelems_and_sum($([]))
@btime addelems_and_sum($(Real[]))
@btime addelems_and_sum($(Float64[]))
@btime addelems_and_sum($(Union{Float64, ComplexF32}[]))
@btime addelems_and_sum($(Union{Float64, Missing}[]))

  5.038 ms (148899 allocations: 2.27 MiB)
  4.853 ms (143499 allocations: 2.19 MiB)
  870.430 μs (0 allocations: 0 bytes)
  1.184 ms (1023 allocations: 15.98 KiB)
  1.193 ms (0 allocations: 0 bytes)


2.06949e7

In [11]:
x = Real[]
@code_warntype addelems_and_sum(x)

MethodInstance for addelems_and_sum(::Vector{Real})
  from addelems_and_sum(arr::AbstractVector) @ Main In[8]:2
Arguments
  #self#::Core.Const(addelems_and_sum)
  arr::Vector{Real}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  i::Float64
Body::Real
1 ─ %1  = (1.0:100.0)::Core.Const(1.0:1.0:100.0)
│         (@_3 = Base.iterate(%1))
│   %3  = (@_3::Core.Const((1.0, 1)) === nothing)::Core.Const(false)
│   %4  = Base.not_int(%3)::Core.Const(true)
└──       goto #4 if not %4
2 ┄ %6  = @_3::Tuple{Float64, Int64}
│         (i = Core.getfield(%6, 1))
│   %8  = Core.getfield(%6, 2)::Int64
│         Main.push!(arr, i)
│         (@_3 = Base.iterate(%1, %8))
│   %11 = (@_3 === nothing)::Bool
│   %12 = Base.not_int(%11)::Bool
└──       goto #4 if not %12
3 ─       goto #2
4 ┄ %15 = Main.sum(arr)::Real
└──       return %15



In [28]:
#do not create data structures with undefined types (abstract containers)
struct MyAmbiguousType
    a
end
struct MyNonambiguousType
    a::Float64
end
function sumarr2(arr)
    res = 0.0
    for e in arr
        res += e.a
    end
    res
end
tt1 = MyAmbiguousType.(1.:10000.)
tt2 = MyNonambiguousType.(1.:10000.)

@btime sumarr2($tt1)
@btime sumarr2($tt2)

  203.500 μs (10000 allocations: 156.25 KiB)
  6.220 μs (0 allocations: 0 bytes)


5.0005e7

In [32]:
struct MyParametrizedType4{T <: Number}  
    a::Union{T, Missing}
end
tt_p = MyParametrizedType3{ComplexF64}.(ComplexF64.(1.0:10_000.0, 0.0))
@btime sumarr2($tt_p)


  16.300 μs (0 allocations: 0 bytes)


5.0005e7 + 0.0im

In [30]:
Val{:bc}

Val{:bc}

In [31]:
tt_miss = MyParametrizedType{Union{Float64, Missing}}.(1.0:1000.0)


1000-element Vector{MyParametrizedType{Union{Missing, Float64}}}:
 MyParametrizedType{Union{Missing, Float64}}(1.0)
 MyParametrizedType{Union{Missing, Float64}}(2.0)
 MyParametrizedType{Union{Missing, Float64}}(3.0)
 MyParametrizedType{Union{Missing, Float64}}(4.0)
 MyParametrizedType{Union{Missing, Float64}}(5.0)
 MyParametrizedType{Union{Missing, Float64}}(6.0)
 MyParametrizedType{Union{Missing, Float64}}(7.0)
 MyParametrizedType{Union{Missing, Float64}}(8.0)
 MyParametrizedType{Union{Missing, Float64}}(9.0)
 MyParametrizedType{Union{Missing, Float64}}(10.0)
 MyParametrizedType{Union{Missing, Float64}}(11.0)
 MyParametrizedType{Union{Missing, Float64}}(12.0)
 MyParametrizedType{Union{Missing, Float64}}(13.0)
 ⋮
 MyParametrizedType{Union{Missing, Float64}}(989.0)
 MyParametrizedType{Union{Missing, Float64}}(990.0)
 MyParametrizedType{Union{Missing, Float64}}(991.0)
 MyParametrizedType{Union{Missing, Float64}}(992.0)
 MyParametrizedType{Union{Missing, Float64}}(993.0)
 MyParametrizedTy

In [11]:
@btime sumarr($tt_p) # parametrized
@btime sumarr($tt_miss)  # parametrized allowing missings

  2.000 ns (0 allocations: 0 bytes)
  610.857 ns (0 allocations: 0 bytes)


<a class="anchor" id="fvariable"></a>
### Do not change the type of a variable within a function
---- [Return to table of contents](#toc) ---

In [33]:
function foo()
    x = 1   # Int
    for i = 1:1_000
        x /= i
    end
    return x
end

function foo2()
    x = 1.0  #Float64
    for i = 1:1_000
        x /= i
    end
    return x
end
@btime foo()
@btime foo2()

  5.567 μs (0 allocations: 0 bytes)
  4.186 μs (0 allocations: 0 bytes)


0.0

<a class="anchor" id="barrier"></a>
### Mitigate type uncertainty with barrier functions
---- [Return to table of contents](#toc) ---

In [51]:
a = Vector{rand(Bool) ? Int64 : Float64}(undef, 4)

4-element Vector{Int64}:
            32
             2
 2040585291600
 2040585291664

In [53]:
function strange_twos(n)
    a = Vector{rand(Bool) ? Int64 : Float64}(undef, n)
    for i = 1:n
       a[i] = 2
    end
    return a
end;
# Let's refactor that to 2 functions
function fill_twos!(a)
   for i = eachindex(a)
       a[i] = 2
   end
end;
function strange_twos2(n)
   a = Vector{rand(Bool) ? Int64 : Float64}(undef, n)
   fill_twos!(a)
   return a
end

strange_twos2 (generic function with 1 method)

In [54]:
@btime strange_twos(1_000);
@btime strange_twos2(1_000);

  24.100 μs (491 allocations: 15.59 KiB)
  1.180 μs (2 allocations: 7.95 KiB)


<a class="anchor" id="column"></a>
### Remember about column-major layout of matrices
---- [Return to table of contents](#toc) ---

In [57]:
@btime vec($x)

  17.854 ns (2 allocations: 80 bytes)


4-element Vector{Int64}:
 1
 3
 2
 4

In [55]:
#note data in memory is aligned along columns
x = [1 2; 3 4]
display(x);
display(vec(x));


2×2 Matrix{Int64}:
 1  2
 3  4

4-element Vector{Int64}:
 1
 3
 2
 4

In [58]:
y  = vec(x)
y[2]=999
x

2×2 Matrix{Int64}:
   1  2
 999  4

In [59]:
xx2 = rand(10_000,10_000);

In [60]:
function sum1(x)
    sum_ = 0.0
    for row in 1:100
        @simd for column in 1:100
            @inbounds sum_ += x[row,column]
        end
    end
    sum_
end

function sum2(x)
    sum_ = 0.0
    for column in 1:100
        @simd for row in 1:100
            @inbounds sum_ += x[row,column]
        end
    end
    sum_
end

@btime sum1($xx2)
@btime sum2($xx2)


  6.560 μs (0 allocations: 0 bytes)
  1.900 μs (0 allocations: 0 bytes)


5001.794683284842

<a class="anchor" id="views"></a>
### Prealocate, use views, vectorize
---- [Return to table of contents](#toc) ---

In [61]:
#preallocate vectors and matrices

function xinc(x)
    return [x, x+1, x+2]
end;

function loopinc()
   y = 0
   for i = 1:10^6
       ret = xinc(i)
       y += ret[2]
   end
   return y
end;

function xinc!(ret::AbstractVector{T}, x::T) where T
    ret[1] = x
    ret[2] = x+1
    ret[3] = x+2
    nothing
end;

function loopinc_prealloc()
   ret = Vector{Int}(undef, 3)
   y = 0
   for i = 1:10^6
       xinc!(ret, i)
       y += ret[2]
   end
   return y
end;

@btime loopinc()
@btime loopinc_prealloc()


  90.745 ms (1000000 allocations: 76.29 MiB)
  470.000 μs (1 allocation: 80 bytes)


500001500000

In [65]:
# if you need subranges of matrices use views - do not copy the data!
fcopy(x) = sum(x[2:end-1]);
fview(x) = sum(@view x[2:end-1]);  
@views fview2(x) = sum(x[2:end-1]);

xvv = rand(10^6);

@btime fcopy(xvv);
@btime fview(xvv);
@btime fview2(xvv);



  2.777 ms (3 allocations: 7.63 MiB)
  230.300 μs (1 allocation: 16 bytes)
  317.500 μs (1 allocation: 16 bytes)


In [66]:
#macro @. can aggregate several vectorization operations

f(x) =       3x.^2 + 4x + 7x.^3;

fdot(x) = @. 3x^2 + 4x + 7x^3

f2(x) =       3 .* x.^2 .+ 4 .* x .+ 7 .* x.^3;


xv = rand(10000)

@btime f(xv)
@btime fdot(xv)
@btime f2(xv);

  58.000 μs (12 allocations: 469.03 KiB)
  8.400 μs (6 allocations: 78.20 KiB)
  5.100 μs (6 allocations: 78.20 KiB)


<a class="anchor" id="warntype"></a>
### Check type stability with the `@code_warntype` macro
---- [Return to table of contents](#toc) ---

In [22]:
pos(x) = x < 0 ? 0 : x;

pos(rand()-0.5)

0

In [23]:
function f(x)
   y = pos(x)
   return sin(y*x + 1)
end;
f(3)

-0.5440211108893698

In [24]:
@code_warntype f(2.3)

MethodInstance for f(::Float64)
  from f(x) @ Main In[23]:1
Arguments
  #self#::Core.Const(f)
  x::Float64
Locals
  y::Union{Float64, Int64}
Body::Float64
1 ─      (y = Main.pos(x))
│   %2 = (y * x)::Float64
│   %3 = (%2 + 1)::Float64
│   %4 = Main.sin(%3)::Float64
└──      return %4



In [25]:
pos(x) = x < 0 ? zero(x) : x;
@code_warntype f(2.3)

MethodInstance for f(::Float64)
  from f(x) @ Main In[23]:1
Arguments
  #self#::Core.Const(f)
  x::Float64
Locals
  y::Float64
Body::Float64
1 ─      (y = Main.pos(x))
│   %2 = (y * x)::Float64
│   %3 = (%2 + 1)::Float64
│   %4 = Main.sin(%3)::Float64
└──      return %4

